<h1><center><span style="color:blue">Particles and decays<br>in the Scikit-HEP project</span></center></h1>

<h2><center>Eduardo Rodrigues<br>University of Cincinnati </center></h2>

<h3><center><span style="color:gray">PyHEP 2019 Workshop, Abingdon, 16-18 October 2019</span></center></h3>

<center><img src="images/Particle_logo.png" alt="Particle package logo" style="width: 200px;"/></center>

<h2><center><span style="color:green">PDG particle data and MC identification codes<br>with the <i>Particle</i> package</span></center></h2>

### **Pythonic interface to**
- Particle Data Group (PDG) particle data table.
- Particle MC identification codes, with inter-MC converters.
- With various extra goodies.

### Package motivation - particle data

- The [PDG](http://pdg.lbl.gov/) provides a <span style="color:green">downloadable table of particle masses, widths, charges and Monte Carlo particle ID numbers</span> (PDG IDs).
  - Most recent file [here](http://pdg.lbl.gov/2019/html/computer_read.html).
- It <span style="color:green">also provided an experimental file with extended information</span>
(spin, quark content, P and C parities, etc.) until 2008 only, see [here](http://pdg.lbl.gov/2008/html/computer_read.html) (not widely known!).

- But <span style="color:green">*anyone* wanting to use these data</span>, the only readily available,
<span style="color:green">has to parse the file programmatically</span>.
- Why not make a Python package to deal with all these data, for everyone?

### Package motivation - MC identification codes

- The <span style="color:green">C++ HepPID and HepPDT libraries provide functions for processing particle ID codes</apan>
in the standard particle (aka PDG) numbering scheme.
- Different event generators have their separate set of particle IDs: Pythia, EvtGen, etc.
- Again, why not make a package providing all functionality/conversions, Python-ically, for everyone?

### Package, in short

- <span style="color:green">Particle</span> - loads extended **PDG data tables** and implements search and manipulations / display.
- <span style="color:green">PDGID</span> - find out as much as possible from the PDG ID number. **No table lookup**.
- <span style="color:green">Converters for MC IDs</span> used in Pythia and Geant.


- Basic usage via the command line.
- Fexible / advanced usage programmatically.

### 1. Command line usage

Search and query ...

In [ ]:
!python -m particle -h

In [ ]:
!python -m particle --version

#### PDGID

Print all information from a PDG ID:

In [ ]:
!python -m particle pdgid 211

#### Particle

Search a particle by its PDG ID - return description summary of particle:

In [ ]:
!python -m particle search 211

Search a particle by its name - either return the description summary of matching particle ...

In [ ]:
!python -m particle search "pi(1400)+"

... or a list of particles matching the keyword in their names:

In [ ]:
!python -m particle search "pi+"

#### Bonus feature: zipapp

Package provides a [zipapp](https://docs.python.org/3/library/zipapp.html) version - **one file** that runs on **any computer with Python**, no other dependencies! Find it [attached to releases](https://github.com/scikit-hep/particle/releases).

Example:

```bash
./particle.pyz search gamma
```

All dependencies (including the two backports) are installed inside the zipapp, and the data lookup is handled in a zip-safe way inside particle. Python 3 is used to make the zipapp, but including the backports makes it work on Python 2 as well.

### 2. `PDGID` class and MC ID classes


- Classes `PDGID`, `PythiaID`, `GeantID`.
- Converters in module `particle.converters`: `Geant2PDGIDBiMap`, etc.

#### PDG IDs module overview

- <span style="color:green">Process and query PDG IDs</span>, and more – no look-up table needed.
  - Current version of package reflects the latest version of the
    <span style="color:green">HepPID & HepPDT utility functions</span> defined in the C++ HepPID and HepPDT versions 3.04.01
  - It contains more functionality than that available in the C++ code … and minor fixes too.
- Definition of a <span style="color:green">PDGID class, PDG ID literals</span>,
and set of standalone HepPID <span style="color:green">functions to query PDG IDs</span>
(is_meson, has_bottom, j_spin, charge, etc.).
   - All PDGID class functions are available standalone.

#### PDGID class
- Wrapper class `PDGID` for PDG IDs.
- Behaves like an int, with extra goodies.
- Large spectrum of properties and methods, with a Pythonic interface, and yet more!

In [ ]:
from particle import PDGID

In [ ]:
pid = PDGID(211)
pid

In [ ]:
PDGID(99999999)

In [ ]:
from particle.pdgid import is_meson

pid.is_meson, is_meson(pid)

To print all `PDGID` properties:

In [ ]:
print(pid.info())

#### MC ID classes and converters

- Classes for MC IDs used in Pythia and Geant: `PythiaID` and `GeantID`.
- Converters in module `particle.converters`: `Geant2PDGIDBiMap`, etc.

In [ ]:
from particle import PythiaID, GeantID

pyid = PythiaID(10221)

pyid.to_pdgid()

Conversions are directly available via mapping classes.

E.g., bi-directional map Pythia ID - PDG ID:

In [ ]:
from particle.converters import Pythia2PDGIDBiMap

Pythia2PDGIDBiMap[PDGID(9010221)]

In [ ]:
Pythia2PDGIDBiMap[PythiaID(10221)]

### 3. `Particle` class

There are lots of ways to create a particle.

In [ ]:
from particle import Particle

#### From a PDG ID

In [ ]:
Particle.from_pdgid(211)

#### From a name

In [ ]:
Particle.from_string('pi+')

#### Searching

Simple and natural API to deal with the PDG particle data table, with powerful search and look-up utilities!

- `Particle.find(…)` – search a single match (exception raised if multiple particles match the search specifications).
- `Particle.findall(…)` – search a list of candidates.

- Powerful search methods that can query any particle property!
- One-line queries.

In [ ]:
Particle.find('J/psi')

You can specify search terms as keywords - _any particle property_:

In [ ]:
Particle.find(latex_name=r'\phi(1020)')

Some properties have enums available. For example, you can directly check the numeric charge:

In [ ]:
Particle.findall('pi', charge=-1)

Or you can use the enum (for charge, this is 3 times the charge, hence the name `three_charge`)

In [ ]:
from particle import Charge

Particle.findall('pi', three_charge=Charge.p)

Or use a **lambda function** for the ultimate in generality! For example, to find all the neutral particles with a bottom quark between 5.2 and 5.3 GeV:

In [ ]:
from hepunits import GeV, s  # Units are good. Use them.

In [ ]:
Particle.findall(lambda p:
                     p.pdgid.has_bottom
                     and p.charge==0
                     and 5.2*GeV < p.mass < 5.3*GeV
                )

Another lambda function example: You can use the width or the lifetime:

In [ ]:
Particle.findall(lambda p: p.lifetime > 1000*s)

If you want infinite lifetime, you could just use the keyword search instead:

In [ ]:
Particle.findall(lifetime=float('inf'))

Trivially find all pseudoscalar charm mesons:

In [ ]:
from particle import SpinType

Particle.findall(lambda p: p.pdgid.is_meson and p.pdgid.has_charm and p.spin_type==SpinType.PseudoScalar)

#### Display

Nice display in Jupyter notebooks, as well as `str` and `repr` support:

In [ ]:
p = Particle.from_pdgid(-435)
p

In [ ]:
print(p)

In [ ]:
print(repr(p))

Full descriptions:

In [ ]:
print(p.describe())

You may find LaTeX or HTML to be more useful in your program; both are supported:

In [ ]:
print(p.latex_name, p.html_name)

It is easy to get hold of the whole list of particle (instances) as a list:

In [ ]:
print('# of particles in loaded data table:', len(Particle.all()))
Particle.all()

#### Particle properties

You can do things to particles, like **invert** them:

In [ ]:
~p

There are a plethora of properties you can access:

In [ ]:
p.spin_type

You can quickly access the PDGID of a particle:

In [ ]:
p.pdgid

In [ ]:
PDGID(p)

### 4. Literals

They provide a <span style="color:green">handy way to manipulate things with human-readable names!</span>

Package defines <span style="color:green">literals for most common particles</span>, with easily recognisable names.
- Literals are dynamically generated on import for both `PDGID` and `Particle` classes.

**PDGID literals**

In [ ]:
from particle.pdgid import literals as lid

In [ ]:
lid.phi_1020

**Particle literals**

In [ ]:
from particle.particle import literals as lpart

In [ ]:
lpart.phi_1020

### 5. Data files

- All data files stored under `particle/data/`

<center><img src="images/PDG_logo.png" alt="PDG logo" style="width: 50px;"/></center>

- <b>PDG particle data files</b>
  - Original PDG data files, which are in a fixed-width format
  - Code uses “digested forms” of the PDG files, stored as CSV, for optimised querying
  - Latest-ish PDG data used by default (2018 at present, 2019 update available since a few days!)
  - Advanced usage: user can load older PDG table, load a “user table” with new particles, append to default table

- Other data files
  - CSV file for mapping of PDG IDs to particle LaTeX names

#### Dump table contents

The `Particle.dump_table(...)` method is rather flexible.

(No need to dig into the package installation directory to inspect the particle data table ;-).)

In [ ]:
help(Particle.dump_table)

In [ ]:
fields = ['pdgid', 'pdg_name', 'mass', 'mass_upper', 'mass_lower', 'three_charge']
    
Particle.dump_table(exclusive_fields=fields, n_rows=10)

Table with all b-flavoured hadrons (in _reStructuredText_ format):

In [ ]:
Particle.dump_table(filter_fn=lambda p: p.pdgid.has_bottom, exclusive_fields=fields, tablefmt='rst')

### 6. Advanced usage

You can:

* Extend or replace the default particle data table in `Particle`.
* Adjust properties for a particle.
* Make custom particles.

<center><img src="images/DecayLanguage_logo.png" alt="DecayLanguage package logo" style="width: 200px;"/></center>

<h2><center><span style="color:green">Decays and decay chains with the <i>DecayLanguage</i> package</span></center></h2>

`DecayLanguage` was designed to support manipulating decay structures in Python. It started with a specific focus, and is slowly generalizing. The current package has:
- Amplitude Analysis decay language:
  - Input based on AmpGen generator
  - Current output formats:
    - GooFit C++
- Decay file parsers:
  - Read DecFiles, such as the LHCb master DecFile
  - Manipulate adn visualize them in Python

### Package motivation

- Ability to describe decay-tree-like structures
- Provide a translation of decay amplitude models from AmpGen to GooFit
  - Idea is to generalise this to other decay descriptions

 - Any experiment uses event generators which, among many things, need to describe particle decay chains
 - Programs such as EvtGen rely on so-called .dec decay files
 - Many experiments need decay data files
 - Why not make a Python package to deal with decay files, for everyone?

### Package, in short

- Tools to parse decay files and programmatically manipulate them, query, display information.
  - Descriptions and parsing built atop the [Lark parser](https://github.com/lark-parser/lark/).
- Tools to translate decay amplitude models from AmpGen to GooFit, and manipulate them.

### 1. Decay files

#### *Master file” DECAY.DEC

<span style="color:green">Gigantic file defining decay modes for all relevant particles, including decay model specifications.</span>

#### User .dec files
- Needed to produce specific MC samples.
- Typically contain a single decay chain (except if defining inclusive samples).

**Example user decay file:**

<pre>
# Decay file for [B_c+ -> (B_s0 -> K+ K-) pi+]cc

Alias      B_c+sig        B_c+
Alias      B_c-sig        B_c-
ChargeConj B_c+sig        B_c-sig
Alias      MyB_s0         B_s0
Alias      Myanti-B_s0    anti-B_s0
ChargeConj MyB_s0         Myanti-B_s0

Decay B_c+sig
  1.000     MyB_s0     pi+     PHOTOS PHSP;
Enddecay
CDecay B_c-sig

Decay MyB_s0
    1.000     K+     K-     SSD_CP 20.e12 0.1 1.0 0.04 9.6 -0.8 8.4 -0.6;
Enddecay
CDecay Myanti-B_s0
</pre>

### 2. Decay file parsing

- **Parsing should be simple**
  - Expert users can configure parser choice and settings, etc. 
- **Parsing should be (reasonably) fast!**

After parsing, many queries are possible!

In [ ]:
from decaylanguage import DecFileParser

#### The LHCb "master decay file"

It's a big file! ~ 450 particle decays defined, thousands of decay modes, over 11k lines in total.

In [ ]:
dfp = DecFileParser('data/DECAY_LHCB.DEC')

In [ ]:
%%time
dfp.parse()

In [ ]:
dfp

Let's parse and play with a small decay file:

In [ ]:
with open('data/Dst.dec') as f:
    print(f.read())

In [ ]:
dfp_Dst = DecFileParser('data/Dst.dec')
dfp_Dst

In [ ]:
dfp_Dst.parse()
dfp_Dst

It can be handy to **parse from a multi-line string** rather than a file:

In [ ]:
s = """
# Decay file for [B_c+ -> (B_s0 -> K+ K-) pi+]cc

Alias      B_c+sig        B_c+
Alias      B_c-sig        B_c-
ChargeConj B_c+sig        B_c-sig
Alias      MyB_s0         B_s0
Alias      Myanti-B_s0    anti-B_s0
ChargeConj MyB_s0         Myanti-B_s0

Decay B_c+sig
  1.000     MyB_s0     pi+     PHOTOS PHSP;
Enddecay
CDecay B_c-sig

Decay MyB_s0
    1.000     K+     K-     SSD_CP 20.e12 0.1 1.0 0.04 9.6 -0.8 8.4 -0.6;
Enddecay
CDecay Myanti-B_s0
"""

dfp = DecFileParser.from_string(s)
dfp.parse()
dfp

#### Decay file information

In [ ]:
dfp_Dst.print_decay_modes('D*+')

In [ ]:
dfp_Dst.list_decay_mother_names()

In [ ]:
dfp_Dst.list_decay_modes('D*+')

#### Info such as particle aliases

In [ ]:
dfp.dict_aliases()

In [ ]:
dfp.dict_charge_conjugates()

### 3.  Display of decay chains

The parser can provide a simple representation of any decay chain found in the input decay file(s).

In [ ]:
dc = dfp_Dst.build_decay_chain('D+')
dc

In [ ]:
from decaylanguage import DecayChainViewer

In [ ]:
DecayChainViewer(dc)

In [ ]:
dc = dfp_Dst.build_decay_chain('D*+')
DecayChainViewer(dc)

In [ ]:
dc = dfp_Dst.build_decay_chain('D*+', stable_particles=['D+', 'D0', 'pi0'])
DecayChainViewer(dc)

#### Charge conjugation:

In [ ]:
dc_cc = dfp_Dst.build_decay_chain('D*-', stable_particles=['D-', 'anti-D0', 'pi0'])
DecayChainViewer(dc_cc)

#### Parsing several files

Typically useful when the user decay file needs information from the master decay file.

In [ ]:
s = u"""
Alias      MyXic+              Xi_c+
Alias      MyantiXic-          anti-Xi_c-
ChargeConj MyXic+              MyantiXic-

Decay Xi_cc+sig
  1.000       MyXic+    pi-    pi+       PHSP;
Enddecay
CDecay anti-Xi_cc-sig

Decay MyXic+
  1.000       p+    K-    pi+       PHSP;
Enddecay
CDecay MyantiXic-

End
"""

dfp = DecFileParser.from_string(s)
dfp.parse()
dfp

Note the subtletly: 3, not 4 decays, are found! This is because the file contains no statement
`ChargeConj anti-Xi_cc-sigXi_cc+sig`, hence the parser cannot know to which particle (matching `Decay` statement) the charge-conjugate decay of `anti-Xi_cc-sig` relates to (code does not rely on position of statements to guess ;-)).

In [ ]:
d = dfp.build_decay_chain('Xi_cc+sig')
DecayChainViewer(d)

As said in the warning, the information provided is not enough for the anti-Xi_cc-sig to make sense:

In [ ]:
from decaylanguage.dec.dec import DecayNotFound

try:
    d = dfp.build_decay_chain('anti-Xi_cc-sig')
except DecayNotFound:
    print("Decays of particle 'anti-Xi_cc-sig' not found in .dec file!")

But the missing information is easily providing **parsing two files simultaneously!** (Any number of files is allowed.)

In [ ]:
from tempfile import NamedTemporaryFile

with NamedTemporaryFile(delete=False) as tf:
    tf.write(s.encode('utf-8'))
    
dfp = DecFileParser(tf.name, 'data/DECAY_LHCB.DEC')
dfp.parse()

In [ ]:
dc = dfp.build_decay_chain('Xi_cc+sig')

DecayChainViewer(dc)

In [ ]:
dc_cc = dfp.build_decay_chain('anti-Xi_cc-sig')

DecayChainViewer(dc_cc)

### 4. Representation of decay chains

Representation of decay chains is of interest well outside the context of decay file parsing!

In [ ]:
from decaylanguage.decay.decay import DecayMode

dm = DecayMode(0.2551,                                              # branching fraction
               'pi- pi0 nu_tau',                                    # final-state particles
               model='TAUHADNU',                                    # decay model
               model_params=[-0.108, 0.775, 0.149, 1.364, 0.400],   # decay-model parameters
               study='toy', year=2019                               # user metadata
              )
dm

In [ ]:
print(dm.describe())

### Interested ? Want to try it ?

#### Particle
- GitHub: https://github.com/scikit-hep/particle/
- Releases: [PyPI](https://pypi.org/project/Particle/)

#### DecayLanguage
- GitHub: https://github.com/scikit-hep/decaylanguage
- Releases: [PyPI](https://pypi.org/project/decaylanguage/)
